In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)
import warnings
warnings.filterwarnings('ignore')

2.11.0


In [2]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [3]:
ds = pd.read_csv('train_data_fixed.csv')
numConversion = ["left", "right", "center", "auth", "liberal"]
#lambda function to convert x to lowercase and strip all spaces
ds['mainlean'] = ds.mainlean.apply(lambda x: numConversion.index(x.lower().strip()))
ds.dtypes
df=ds.sample(frac=1)

In [4]:
df.head(1)

,tweet,mainlean,secondlean
84696,Throughout the pandemic nurses have put their ...,0,NaN


In [5]:
split = round(len(df)*0.8)
train_tweets = df['tweet'][:split]
train_label = df['mainlean'][:split]
test_tweets = df['tweet'][split:]
test_label = df['mainlean'][split:]

In [6]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for row in train_tweets:
    training_sentences.append(str(row))
for row in train_label:
    training_labels.append((row))
for row in test_tweets:
    testing_sentences.append(str(row))
for row in test_label:
    testing_labels.append((row))

In [7]:
print((training_labels[0]))

0


In [8]:
vocab_size = 60000
embedding_dim = 18
max_length = 280
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [10]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)

In [25]:
tf.keras.backend.clear_session()
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=int(max_length)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(15, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 280, 18)           1080000   
                                                                 
 global_average_pooling1d (G  (None, 18)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 15)                285       
                                                                 
 dropout (Dropout)           (None, 15)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 80        
                                                                 
Total params: 1,080,365
Trainable params: 1,080,365
Non-trainable params: 0
______________________________________________

In [26]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
poliIdent = model.fit(padded, training_labels_final, epochs=10, validation_data=(testing_padded, testing_labels_final), callbacks=[es_callback])

Epoch 1/10
3525/3525 [==============================] - 28s 8ms/step - loss: 0.7656 - accuracy: 0.4965 - val_loss: 0.7392 - val_accuracy: 0.4975
Epoch 2/10
3525/3525 [==============================] - 27s 8ms/step - loss: 0.7008 - accuracy: 0.5831 - val_loss: 0.5290 - val_accuracy: 0.7617
Epoch 3/10
3525/3525 [==============================] - 27s 8ms/step - loss: 0.4695 - accuracy: 0.7935 - val_loss: 0.4336 - val_accuracy: 0.8116
Epoch 4/10
3525/3525 [==============================] - 28s 8ms/step - loss: 0.3895 - accuracy: 0.8388 - val_loss: 0.4012 - val_accuracy: 0.8308
Epoch 5/10
3525/3525 [==============================] - 28s 8ms/step - loss: 0.3422 - accuracy: 0.8647 - val_loss: 0.3754 - val_accuracy: 0.8444
Epoch 6/10
3525/3525 [==============================] - 28s 8ms/step - loss: 0.3102 - accuracy: 0.8811 - val_loss: 0.3630 - val_accuracy: 0.8510
Epoch 7/10
3525/3525 [==============================] - 28s 8ms/step - loss: 0.2838 - accuracy: 0.8941 - val_loss: 0.3600 - val_ac

In [27]:
model.layers[0].get_weights()[0]

array([[-0.08538548, -0.00694711, -0.006958  , ...,  0.07766363,
         0.01360046,  0.01135165],
       [-0.4605619 ,  0.3688638 , -0.15768583, ..., -0.35691035,
        -0.07755852,  0.2795269 ],
       [-0.5378631 ,  0.5442894 , -0.02868105, ..., -0.17634577,
         0.10809467,  0.49746355],
       ...,
       [ 0.14734583,  0.18665616, -0.2020453 , ...,  0.16109364,
        -0.23854694,  0.14185299],
       [-0.20849594, -0.22215609,  0.19003974, ..., -0.27185673,
         0.23192607, -0.22810994],
       [ 0.17166565,  0.17356603, -0.23084834, ...,  0.17090242,
        -0.1864707 ,  0.17099053]], dtype=float32)

In [28]:
model.save("poliIdent")

INFO:tensorflow:Assets written to: poliIdent\assets


INFO:tensorflow:Assets written to: poliIdent\assets
